# Lab 4

## JMuBEN2 dataset


(https://data.mendeley.com/datasets/tgv3zb82nd/1)

Image dataset taken from an Arabica coffee plantation. The image dataset contains collection of healthy and miner coffee leaves.

### Loading and Spliting the dataset 

In [1]:
import os
import requests
import zipfile
import numpy as np

dataset_url = 'https://github.com/ishaqmarashy/DATALFS/raw/main/JMuBEN.zip'
dataset_dir = './JMuBEN'

# create directory for dataset if it does not exist
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    
# append JMuBEN.zip to the end of the path (this is where we download the file to)
zip_file_path = os.path.join(dataset_dir, 'JMuBEN.zip')


# check if file is downloaded already
if not os.path.exists(zip_file_path):
    
    # file is not downloaded so fetch the file
    response = requests.get(dataset_url)
    
    # write file to storage which is recieved from the response
    with open(zip_file_path, 'wb') as zip_file:
        zip_file.write(response.content)
        
    # unzip to zip file path
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_dir)

# within the concat train and test to become ./JMuBEN/train and JMuBEN ./JMuBEN/test
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

In [2]:
def load_images_and_labels(directory):
    images = []
    labels = []
    
    # get subdirectories Healthy and Miner
    
    for class_name in os.listdir(directory):
        
        # concat subdirectory to get full path
        class_dir = os.path.join(directory, class_name)
        # assign labels using class subdirectory
        # label is determined by filepath
        label = 0 if class_name == 'Miner' else 1
        
        # append labels and image paths to labels and images respectively
        for filename in os.listdir(class_dir):
            images.append(os.path.join(class_dir, filename))
            labels.append(label)
            
    return images, labels

# load file directories and their labels
train_images_dir, train_labels = load_images_and_labels(train_dir)
test_images_dir, test_labels = load_images_and_labels(test_dir)

In [3]:
# print the number of images and labels

print(f"Train images:{len(train_images_dir)}  Labels:{len(train_labels)}")
print(f"Test images:{len(test_images_dir)}  Labels:{len(test_labels)}")


Train images:24000  Labels:24000
Test images:6000  Labels:6000


### Pre-processing and Normalization

Images in the dataset were converted to grayscale and resized to 128x128

In [4]:
import cv2

def load_normalize_images(image_paths):
    images = []
    for image_path in image_paths:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  #  grayscale
        img = cv2.resize(img, (128, 128))  # resize
        img = img / 255.0  # normalize
        images.append(img)
    return np.array(images)

train_images=load_normalize_images(train_images_dir)
test_images=load_normalize_images(test_images_dir)



In [5]:
train_images[0].shape

(128, 128)

### Model Training

In [6]:
import pandas as pd
from sklearn.metrics import classification_report
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

In [10]:
from keras.optimizers import Adam 


model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(32, (3, 3),padding='same', activation="relu", input_shape=(128, 128,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),    
    tf.keras.layers.Conv2D(16, (3, 3),padding='same', activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),   
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation="softmax")
    
])

model.compile(optimizer = Adam(0.0001), loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(np.asarray(train_images, dtype=np.float32), np.asarray(train_labels), batch_size=128, epochs=50, validation_split=0.2, shuffle=True)


Epoch 1/50
150/150 [==============================] - 81s 540ms/step - loss: 0.5467 - accuracy: 0.6916 - val_loss: 0.5574 - val_accuracy: 0.8102
Epoch 2/50
150/150 [==============================] - 78s 522ms/step - loss: 0.4310 - accuracy: 0.7790 - val_loss: 0.6757 - val_accuracy: 0.6417
Epoch 3/50
150/150 [==============================] - 74s 492ms/step - loss: 0.3475 - accuracy: 0.8290 - val_loss: 0.4392 - val_accuracy: 0.8117
Epoch 4/50
150/150 [==============================] - 78s 519ms/step - loss: 0.2899 - accuracy: 0.8493 - val_loss: 0.2867 - val_accuracy: 0.8833
Epoch 5/50
150/150 [==============================] - 79s 525ms/step - loss: 0.2554 - accuracy: 0.8595 - val_loss: 0.2245 - val_accuracy: 0.9115
Epoch 6/50
150/150 [==============================] - 73s 487ms/step - loss: 0.2236 - accuracy: 0.8704 - val_loss: 0.2808 - val_accuracy: 0.8821
Epoch 7/50
150/150 [==============================] - 73s 488ms/step - loss: 0.2108 - accuracy: 0.8757 - val_loss: 0.1748 - val_ac

In [14]:
train_images = train_images.reshape(train_images.shape[0], -1)

### SGD Classifier 

In [11]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
sgd_clf.fit(train_images, train_labels)

ValueError: Found array with dim 3. SGDClassifier expected <= 2.

In [ ]:
test_images = test_images.reshape(test_images.shape[0], -1)

In [ ]:
sgd_clf.predict(test_images)

### Model Evaluation
